<a href="https://colab.research.google.com/github/JisuEun/presentation_oss/blob/main/AI_Chatbot_using_KR_SBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=7b15bdb433591824c5059683612615313d74bef587deb57e8ad4152099674cd8
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/chatbot_data/data3.csv', encoding='EUC-KR')
train_data.head()

,question,answer
0,유리노를 착용한 채로 운동 가능한가?,운동 중에 유리노 착용을 추천합니다. 복압성 요실금을 완화하여 안심하고 운동할 수 ...
1,치료 중에 유리노를 입고 운동하는 것이 가능한가요?,운동 중에 유리노 착용을 추천합니다. 복압성 요실금을 완화하여 안심하고 운동할 수 ...
2,유리노를 착용하면서도 운동할 수 있는지 알려주세요.,운동 중에 유리노 착용을 추천합니다. 복압성 요실금을 완화하여 안심하고 운동할 수 ...
3,운동할 때 유리노를 사용하는 건 괜찮아?,운동 중에 유리노 착용을 추천합니다. 복압성 요실금을 완화하여 안심하고 운동할 수 ...
4,유리노를 착용한 채로도 운동 가능한가요?,운동 중에 유리노 착용을 추천합니다. 복압성 요실금을 완화하여 안심하고 운동할 수 ...


In [ ]:
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.question), axis = 1)

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)

    uncertainty = 1
    for i in range(0, train_data.shape[0]):
      row = train_data.iloc[i]
      if row['score'] > 0.4:
        uncertainty = 0

    print(train_data['score'])
    if uncertainty == 1:
      return '질문하신 내용을 이해하지 못했습니다.'
    return train_data.loc[train_data['score'].idxmax()]['answer']

In [ ]:
return_answer('운동할 때 유리노 착용해도 돼?')

0      0.814749
1      0.721243
2      0.755520
3      0.876754
4      0.818418
         ...   
310    0.240039
311    0.203305
312    0.206048
313    0.305536
314    0.309696
Name: score, Length: 315, dtype: float32


'운동 중에 유리노 착용을 추천합니다. 복압성 요실금을 완화하여 안심하고 운동할 수 있습니다.'

In [ ]:
return_answer('음')

0      0.142940
1      0.158248
2      0.093925
3      0.182594
4      0.109611
         ...   
310    0.003786
311    0.028256
312    0.094044
313    0.122518
314    0.119713
Name: score, Length: 315, dtype: float32


'질문하신 내용을 이해하지 못했습니다.'

In [ ]:
import torch
torch.save(obj=model, f='test.pt')